In [7]:
import torch as T
import torch.nn as nn

In [35]:
class CovertypeClass(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(54, 128)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.2)
        self.layer2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.drop2 = nn.Dropout(p=0.2)
        self.layer3 = nn.Linear(128, 64)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(64, 7)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.drop1(x)
        x = self.act2(self.layer2(x))
        x = self.drop2(x)
        x = self.act3(self.layer3(x))
        x = self.output(x)
        x = self.softmax(x)
        return x

In [9]:
from sklearn.datasets import fetch_covtype

covtype = fetch_covtype()
X = covtype.data
y = covtype.target

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [11]:
type(X_train)

numpy.ndarray

In [12]:
from torch.utils.data import TensorDataset, DataLoader

X_train_t = T.tensor(X_train).float()
X_test_t = T.tensor(X_test).float()
y_train_t = T.tensor(y_train).long().reshape(-1)
y_train_t = y_train_t - 1
y_test_t = T.tensor(y_test).long().reshape(-1)
y_test_t = y_test_t - 1
print(y_train_t.unique())

tensor([0, 1, 2, 3, 4, 5, 6])


In [13]:
print(X_train_t.shape)
print(y_train_t.shape)

torch.Size([464809, 54])
torch.Size([464809])


In [14]:
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [16]:
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu")
print(device)

cpu


In [17]:
net = CovertypeClass().to(device)

In [18]:
import numpy as np

In [19]:
from sklearn.metrics import precision_score, recall_score

In [20]:
def test_model(net, loss_fn, test_loader):
    total_loss = 0
    net.eval()

    with T.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            preds = net(x)
            preds_max = np.argmax(preds, axis = 1)
            loss = loss_fn(preds, y)
            total_loss += loss.item()
            recall = recall_score(y, preds_max, average = 'weighted', zero_division = 0)
            precision = precision_score(y, preds_max, average = 'weighted', zero_division = 0)

    print(total_loss / len(test_loader))
    print(recall)
    print(precision)

In [21]:
optimizer = T.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [22]:
test_model(net, loss_fn, test_loader)

2.129789435390859
0.0
0.0


In [23]:
EPOCHS = 50

In [24]:
net.train()
for i in range(EPOCHS):
    if i % 5 == 0:
        print(f"Running epoch: {i}")

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Running epoch: 0
Running epoch: 5
Running epoch: 10
Running epoch: 15
Running epoch: 20
Running epoch: 25
Running epoch: 30
Running epoch: 35
Running epoch: 40
Running epoch: 45


In [25]:
test_model(net, loss_fn, test_loader)

1.6798833044418153
0.7272727272727273
0.5289256198347108


In [26]:
best_loss = float('inf')
best_epoch = 0
tolerance = 0
max_tolerance = 2

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train_t, y_train_t, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)


train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32)

In [29]:
import torch
for epoch in range(EPOCHS):
    train_loss = 0.0
    val_loss = 0.0
    net.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    net.eval()
    with torch.no_grad():
        for x_val, y_val in val_loader:
            preds_val = net(x_val)
            loss_val = loss_fn(preds_val, y_val)
            val_loss += loss.item()


    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")


    if val_loss < best_loss:
        best_loss = val_loss
        best_epoch = epoch
        tolerance = 0
    else:
        tolerance += 1


    if tolerance > max_tolerance:
        print(f"Early stopping at epoch {epoch+1}")
        break

Epoch 1: Train Loss=1.6916, Val Loss=2.0226
Epoch 2: Train Loss=1.6773, Val Loss=1.4511
Epoch 3: Train Loss=1.7086, Val Loss=1.6186
Epoch 4: Train Loss=1.7680, Val Loss=1.7565
Epoch 5: Train Loss=1.6737, Val Loss=1.5940
Early stopping at epoch 5


In [30]:
learning_rate = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
size_of_hidden_layers = [64, 128, 256]

best_precision = 0.0
best_model = None

In [31]:
best_loss = float('inf')
best_epoch = 0
tolerance = 0
max_tolerance = 2

In [36]:
EPOCHS = 30


for lr in learning_rate:
    for size in size_of_hidden_layers:
        for batch_size in batch_sizes:
            model = CovertypeClass(54, size, 7)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)

            for epoch in range(EPOCHS):
                train_loss = 0.0
                val_loss = 0.0
                net.train()
                for x, y in train_loader:
                    x = x.to(device)
                    y = y.to(device)
                    preds = net(x)
                    loss = loss_fn(preds, y)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()

                net.eval()
                with torch.no_grad():
                    for x_val, y_val in val_loader:
                        preds_val = net(x_val)
                        loss_val = loss_fn(preds_val, y_val)
                        val_loss += loss.item()

                precision = precision_score(y_val, torch.argmax(preds_val, dim = 1), average = 'weighted', zero_division = 0)
                print(precision)

                if precision > best_precision:
                    best_precision = precision
                    best_model = net
                    best_lr = lr
                    best_size = size
                    best_batch_size = batch_size

                train_loss /= len(train_loader)
                val_loss /= len(val_loader)

                print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")

                if val_loss < best_loss:
                    best_loss = val_loss
                    best_epoch = epoch
                    tolerance = 0
                else:
                    tolerance += 1

                if tolerance > max_tolerance:
                    print(f"Early stopping at epoch {epoch+1}")
                    best_loss = float('inf')
                    best_epoch = 0
                    tolerance = 0
                    max_tolerance = 2
                    break



print("Best Precision:", best_precision)
print("Best Model:", best_model)
print("Best learning rate:", best_lr)
print("Best size of hidden layers:", best_size)
print("Best batch size:", best_model)

0.25
Epoch 1: Train Loss=1.6760, Val Loss=1.6231
0.25
Epoch 2: Train Loss=1.6760, Val Loss=1.7016
0.25
Epoch 3: Train Loss=1.6760, Val Loss=1.7369
0.25
Epoch 4: Train Loss=1.6760, Val Loss=1.5940
0.25
Epoch 5: Train Loss=1.6760, Val Loss=1.4664
0.25
Epoch 6: Train Loss=1.6757, Val Loss=1.5284
0.25
Epoch 7: Train Loss=1.6760, Val Loss=1.8797
0.25
Epoch 8: Train Loss=1.6760, Val Loss=1.7277
Early stopping at epoch 8
0.25
Epoch 1: Train Loss=1.6762, Val Loss=1.7453
0.25
Epoch 2: Train Loss=1.6759, Val Loss=1.8797
0.25
Epoch 3: Train Loss=1.6758, Val Loss=1.4094
0.25
Epoch 4: Train Loss=1.6762, Val Loss=1.6157
0.25
Epoch 5: Train Loss=1.6760, Val Loss=1.7546
0.25
Epoch 6: Train Loss=1.6758, Val Loss=1.4431
Early stopping at epoch 6
0.25
Epoch 1: Train Loss=1.6758, Val Loss=1.4511
0.25
Epoch 2: Train Loss=1.6758, Val Loss=2.0226
0.25
Epoch 3: Train Loss=1.6758, Val Loss=1.5835
0.25
Epoch 4: Train Loss=1.6762, Val Loss=1.8731
Early stopping at epoch 4
0.25
Epoch 1: Train Loss=1.6760, Val Los